In [ ]:
from ruptura.RupturaTable import RupturaTable
from ruptura.CreateBatch import CreateBatch
import collections

In [ ]:
rupturaTable = RupturaTable()
files = [
    'ruptura-2018-dezembro-1oQ.csv', 
    'ruptura-2018-dezembro-2oQ.csv', 
    'ruptura-2019-fevereiro.csv', 
    'ruptura-2019-janeiro-1oQ.csv', 
    'ruptura-2019-janeiro-2oQ.csv', 
    'ruptura-2019-marco.csv',
    'ruptura-2019-marco-09-ate-15.csv']
clipData = ['01/11/2018', '01/04/2019']  # mais recente 08/03/2019
rupturaTable.generate(files,clipData)

In [ ]:
createBatch = CreateBatch()
amostras = createBatch.create(rupturaTable.getData(), 'BARBIERI', '01/03/2019')
createBatch.exportBatch(amostras, 'barbieri-interno-1-3-19.json')

In [ ]:
# eu vou vir aqui e definir uma nova forma de gerar esses dados
# seria interessante ter um objeto - data encoder em que ele recebesse a versao e fizesse o encoder correto.
# a primeira que eu quero definir e
# - input - igual estava antes pra comecar
# - saida [ruptura, presenca, desconhecido]

In [ ]:
createBatch = CreateBatch()
amostras = createBatch.createItem(rupturaTable.getData(), 'BISCOITO BARBIERI PAPA OVO 150G', '01/03/2019')

# GERANDO TABELA PARA CONFERENCIA

In [ ]:
# APENAS BARBIERI
indBarbieri = []
for i in data.index:
    if data.Produto[i] in todosBiscoitos:
        indBarbieri.append(i)
dataBarbieri = data.iloc[indBarbieri,:].copy()
dataBarbieri = dataBarbieri.reset_index(drop=True)
dataBarbieri.to_csv('data-barbieri.csv',sep=';')